In [1]:
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torchtext.datasets.imdb import IMDB
import sys

In [2]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [3]:
is_cuda = torch.cuda.is_available()

In [4]:
TEXT = data.Field(lower=True, fix_length=200, batch_first=True)
LABEL = data.Field(sequential=False)

In [5]:
train, test = IMDB.splits(TEXT, LABEL)

In [6]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(train)

In [7]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device='cuda')
train_iter.repeat = False
test_iter.repeat = False

In [8]:
batch = next(iter(train_iter))

In [9]:
n_vocab = len(TEXT.vocab)
hidden_size = 100

In [10]:
class IMDBCnn(nn.Module):
    
    def __init__(self,vocab,hidden_size,n_cat,bs=1,kernel_size=3,max_len=200):
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = bs
        self.e = nn.Embedding(n_vocab,hidden_size)
        self.cnn = nn.Conv1d(max_len,hidden_size,kernel_size)
        self.avg = nn.AdaptiveAvgPool1d(10)
        self.fc = nn.Linear(1000,n_cat)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self,inp):
        bs = inp.size()[0]
        if bs != self.bs:
            self.bs = bs
        e_out = self.e(inp)
        cnn_o = self.cnn(e_out) 
        cnn_avg = self.avg(cnn_o)
        cnn_avg = cnn_avg.view(self.bs,-1)
        fc = F.dropout(self.fc(cnn_avg),p=0.5)
        return self.softmax(fc)


In [11]:
model = IMDBCnn(n_vocab,hidden_size,n_cat=3,bs=32,kernel_size=2)
model = model.cuda()

optimizer = optim.Adam(model.parameters(),lr=1e-3)

def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.cuda(),target.cuda()
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target, reduction='sum').data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct.item()/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [12]:
%%time
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,10):
    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is  0.85 and training accuracy is 11514/25000     46.06
validation loss is  0.76 and validation accuracy is 13166/25000     52.66
training loss is  0.68 and training accuracy is 14276/25000      57.1
validation loss is  0.71 and validation accuracy is 13793/25000     55.17
training loss is  0.58 and training accuracy is 15122/25000     60.49
validation loss is  0.67 and validation accuracy is 14411/25000     57.64
training loss is  0.54 and training accuracy is 15608/25000     62.43
validation loss is  0.68 and validation accuracy is 14569/25000     58.28
training loss is  0.51 and training accuracy is 15740/25000     62.96
validation loss is  0.72 and validation accuracy is 14450/25000      57.8
training loss is  0.47 and training accuracy is 16190/25000     64.76
validation loss is  0.75 and validation accuracy is 14472/25000     57.89
training loss is  0.44 and training accuracy is 16392/25000     65.57
validation loss is  0.83 and validation accuracy is 14370/25000   